In [1]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time

### Class for the Neural Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, activation_function, apply_softmax = False):
        super(NN, self).__init__()

        seed = 18
        torch.manual_seed(seed)

        self.activation = activation_function
        self.apply_softmax = apply_softmax

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_layer_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1])
            for i in range(len(hidden_layer_sizes) - 1)
        ])

        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_classes)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


    def forward(self, x):
        x = x.float()
        x = self.activation(self.input_layer(x))

        # Process through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))

        if self.apply_softmax:
            x = F.softmax(self.output_layer(x), dim=1)
        else:
            x = self.output_layer(x)

        return x
    

### Function to train the model

In [3]:
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0  # Initialize total loss for the epoch
        num_batches = len(train_loader)

        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the batch loss

        average_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")
    

### Function to calculate the accuracy

In [4]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


### Function to split the data into train and test

In [5]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y

## Function for Random Search

In [6]:
def random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = initial_configuration
    train_times = []
    results = []

    for _ in range(num_iterations):
        # Randomly sample hyperparameter values inthe specified ranges
        sampled_configuration = {
            'Hidden Layer Sizes': [random.randint(param_ranges['min_hidden'], param_ranges['max_hidden']) for _ in range(random.randint(param_ranges['min_layers'], param_ranges['max_layers']))],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': random.uniform(param_ranges['min_lr'], param_ranges['max_lr']),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': random.choice(param_ranges['num_epochs'])
        }

        train_start_time = time.time()
        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                   num_classes=num_classes,
                   hidden_layer_sizes=sampled_configuration['Hidden Layer Sizes'],
                   activation_function=sampled_configuration['Activation Function'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=sampled_configuration['Learning Rate'])

        train_model(model, train_loader, optimizer, criterion, sampled_configuration['Number of Epochs'])

        train_end_time = time.time()
        train_times.append(train_end_time - train_start_time)
        accuracy_test = check_accuracy(test_loader, model)

        result_entry = {
            'Hidden Layer Sizes': sampled_configuration['Hidden Layer Sizes'],
            'Activation Function': sampled_configuration['Activation Function'].__name__,
            'Learning Rate': sampled_configuration['Learning Rate'],
            'Batch Size': sampled_configuration['Batch Size'],
            'Number of Epochs': sampled_configuration['Number of Epochs'],
            'Accuracy': accuracy_test.item(),
            'Training Time': train_end_time - train_start_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = sampled_configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

### Random Search that iteratively improves current configuration

In [7]:
# takes way too long
def iterative_random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader, max_attempts=10):
    best_accuracy = 0.0
    best_combination = initial_configuration
    train_times = []
    results = []

    iteration = 0
    while iteration < num_iterations:
        attempt = 0
        found_improvement = False

        while attempt < max_attempts:
            sampled_configuration = {
                'Hidden Layer Sizes': [random.randint(param_ranges['min_hidden'], param_ranges['max_hidden']) for _ in range(random.randint(param_ranges['min_layers'], param_ranges['max_layers']))],
                'Activation Function': random.choice(param_ranges['activation_functions']),
                'Learning Rate': random.uniform(param_ranges['min_lr'], param_ranges['max_lr']),
                'Batch Size': random.choice(param_ranges['batch_sizes']),
                'Number of Epochs': random.choice(param_ranges['num_epochs'])
            }

            train_start_time = time.time()
            model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                       num_classes=num_classes,
                       hidden_layer_sizes=sampled_configuration['Hidden Layer Sizes'],
                       activation_function=sampled_configuration['Activation Function'])
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=sampled_configuration['Learning Rate'])

            train_model(model, train_loader, optimizer, criterion, sampled_configuration['Number of Epochs'])

            train_end_time = time.time()
            train_times.append(train_end_time - train_start_time)
            accuracy_test = check_accuracy(test_loader, model)

            result_entry = {
                'Hidden Layer Sizes': sampled_configuration['Hidden Layer Sizes'],
                'Activation Function': sampled_configuration['Activation Function'].__name__,
                'Learning Rate': sampled_configuration['Learning Rate'],
                'Batch Size': sampled_configuration['Batch Size'],
                'Number of Epochs': sampled_configuration['Number of Epochs'],
                'Accuracy': accuracy_test.item(),
                'Training Time': train_end_time - train_start_time
            }

            results.append(result_entry)

            if accuracy_test > best_accuracy:
                best_accuracy = accuracy_test
                best_combination = sampled_configuration
                found_improvement = True
                break  # Exit the attempt loop if an improvement is found

            attempt += 1

        # Increment iteration only if an improvement is found or max attempts reached
        iteration += 1 if found_improvement else 0

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Function for Local Search

In [8]:
def local_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = initial_configuration
    current_configuration = initial_configuration
    train_times = []
    results = []

    for _ in range(num_iterations):
        # Small changes to the current configuration
        new_configuration = {
            'Hidden Layer Sizes': [
                max(1, size + random.randint(-1, 1)) for size in current_configuration['Hidden Layer Sizes']
            ],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': max(param_ranges['min_lr'], min(param_ranges['max_lr'], current_configuration['Learning Rate'] + random.uniform(-0.01, 0.01))),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': max(1, current_configuration['Number of Epochs'] + random.randint(-1, 1))
        }

        train_start_time = time.time()
        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                   num_classes=num_classes,
                   hidden_layer_sizes=new_configuration['Hidden Layer Sizes'],
                   activation_function=new_configuration['Activation Function'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=new_configuration['Learning Rate'])

        train_model(model, train_loader, optimizer, criterion, new_configuration['Number of Epochs'])

        train_end_time = time.time()
        train_times.append(train_end_time - train_start_time)
        accuracy_test = check_accuracy(test_loader, model)

        result_entry = {
            'Hidden Layer Sizes': new_configuration['Hidden Layer Sizes'],
            'Activation Function': new_configuration['Activation Function'].__name__,
            'Learning Rate': new_configuration['Learning Rate'],
            'Batch Size': new_configuration['Batch Size'],
            'Number of Epochs': new_configuration['Number of Epochs'],
            'Accuracy': accuracy_test.item(),
            'Training Time': train_end_time - train_start_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = new_configuration
            current_configuration = new_configuration  # Update the current configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Loading / preparing data

### Wine Quality

In [9]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


### Congressional Voting

In [10]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


### Bank Marketing

In [11]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)
bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


# Test Random Search over all three datasets

In [19]:
datasets = {'wine_quality': wine_quality, 'cong_voting': cong_voting, 'bank_marketing': bank_marketing}
num_iterations = 10
param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

initial_configuration = {
    'Hidden Layer Sizes': [25],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 64,
    'Number of Epochs': 10
}



all_random_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))

    best_config, best_accuracy, random_results = random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)

    random_results['dataset'] = dataset_name
    all_random_results.append(random_results)



Epoch 1/10, Average Loss: 1.3197862019568134
Epoch 2/10, Average Loss: 1.2889535427093506
Epoch 3/10, Average Loss: 1.2803216631427132
Epoch 4/10, Average Loss: 1.2810276790630597
Epoch 5/10, Average Loss: 1.2814096929105514
Epoch 6/10, Average Loss: 1.2738068937523965
Epoch 7/10, Average Loss: 1.2703259927363484
Epoch 8/10, Average Loss: 1.267993262209044
Epoch 9/10, Average Loss: 1.2744909290887096
Epoch 10/10, Average Loss: 1.2766637630257869
Epoch 1/10, Average Loss: 1.452794024549379
Epoch 2/10, Average Loss: 1.4252949104718635
Epoch 3/10, Average Loss: 1.4237999967270834
Epoch 4/10, Average Loss: 1.422739256379063
Epoch 5/10, Average Loss: 1.423181538932894
Epoch 6/10, Average Loss: 1.4235243980138579
Epoch 7/10, Average Loss: 1.4232048703117606
Epoch 8/10, Average Loss: 1.4233405356026867
Epoch 9/10, Average Loss: 1.4218687133555032
Epoch 10/10, Average Loss: 1.4234526325588577
Epoch 1/5, Average Loss: 2.537899388126069
Epoch 2/5, Average Loss: 1.3752946970652948
Epoch 3/5, Aver

Epoch 9/15, Average Loss: 0.35221451421819844
Epoch 10/15, Average Loss: 0.35177093673388937
Epoch 11/15, Average Loss: 0.3513098816414481
Epoch 12/15, Average Loss: 0.3510283025506052
Epoch 13/15, Average Loss: 0.35099822204668546
Epoch 14/15, Average Loss: 0.3510009241769615
Epoch 15/15, Average Loss: 0.3510019266706647
Epoch 1/5, Average Loss: 0.3542980892859508
Epoch 2/5, Average Loss: 0.35156082327079785
Epoch 3/5, Average Loss: 0.3515387743159912
Epoch 4/5, Average Loss: 0.3516518523025968
Epoch 5/5, Average Loss: 0.351641311665581
Epoch 1/10, Average Loss: 0.3552796747038159
Epoch 2/10, Average Loss: 0.3516843137177593
Epoch 3/10, Average Loss: 0.35180559665986416
Epoch 4/10, Average Loss: 0.3518283847665179
Epoch 5/10, Average Loss: 0.3518333936815412
Epoch 6/10, Average Loss: 0.3518345088924019
Epoch 7/10, Average Loss: 0.3518347536173007
Epoch 8/10, Average Loss: 0.35183477166416216
Epoch 9/10, Average Loss: 0.3518347689352966
Epoch 10/10, Average Loss: 0.3518347660111245
Epo

In [20]:
random_results_df = pd.concat(all_random_results, ignore_index=True)
random_results_df = pd.DataFrame(all_random_results)
#random_results_df = pd.concat(all_random_results, ignore_index=True)

In [21]:
random_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time,dataset
0,"[36, 32, 38]",tanh,0.009886,64,10,0.475817,2.155967,wine_quality
1,"[48, 36, 47]",tanh,0.047072,128,10,0.284967,2.258070,wine_quality
2,"[25, 40]",relu,0.003959,128,5,0.380392,1.185348,wine_quality
3,"[13, 21]",tanh,0.013116,64,5,0.458824,1.714061,wine_quality
4,"[28, 41]",relu,0.007498,128,15,0.501961,5.354725,wine_quality
5,"[6, 13, 26]",sigmoid,0.052761,32,15,0.458824,4.421109,wine_quality
6,"[21, 22]",sigmoid,0.060647,32,10,0.458824,1.939128,wine_quality
7,"[6, 5, 32]",relu,0.016505,32,5,0.458824,0.914510,wine_quality
8,[39],tanh,0.050208,128,5,0.284967,0.811650,wine_quality
9,"[22, 42]",relu,0.061699,64,10,0.492810,3.575577,wine_quality


### Testing Local search

In [24]:
dataset = cong_voting

train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

train_data = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

test_data = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

input_size = train_X.shape[1]

num_classes = 2

initial_configuration = {
    'Hidden Layer Sizes': [25, 30],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 32,
    'Number of Epochs': 10
}

param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

num_iterations = 50

best_combination, best_accuracy, results_df_local = local_search(
    num_iterations, initial_configuration, param_ranges, train_loader, test_loader
)



Epoch 1/10, Average Loss: 52.071486473083496
Epoch 2/10, Average Loss: 8.601516564687094
Epoch 3/10, Average Loss: 13.125688791275024
Epoch 4/10, Average Loss: 4.817462533712387
Epoch 5/10, Average Loss: 2.8842486888170242
Epoch 6/10, Average Loss: 1.6355683853228886
Epoch 7/10, Average Loss: 1.2476839224497478
Epoch 8/10, Average Loss: 0.819334497054418
Epoch 9/10, Average Loss: 0.42724748452504474
Epoch 10/10, Average Loss: 0.5756969600915909
Epoch 1/11, Average Loss: 0.675984799861908
Epoch 2/11, Average Loss: 0.6627763013044993
Epoch 3/11, Average Loss: 0.6622703870137533
Epoch 4/11, Average Loss: 0.6570416589577993
Epoch 5/11, Average Loss: 0.6547190447648367
Epoch 6/11, Average Loss: 0.651234487692515
Epoch 7/11, Average Loss: 0.648522675037384
Epoch 8/11, Average Loss: 0.6439297298590342
Epoch 9/11, Average Loss: 0.6326681872208914
Epoch 10/11, Average Loss: 0.6039976676305135
Epoch 11/11, Average Loss: 0.5593212842941284
Epoch 1/11, Average Loss: 0.8183950384457906
Epoch 2/11, 

Epoch 6/10, Average Loss: 4.367609649896622
Epoch 7/10, Average Loss: 2.1330124835173288
Epoch 8/10, Average Loss: 1.5242833246787388
Epoch 9/10, Average Loss: 1.0582702706257503
Epoch 10/10, Average Loss: 0.32632433871428174
Epoch 1/10, Average Loss: 0.6854649285475413
Epoch 2/10, Average Loss: 0.6647487084070841
Epoch 3/10, Average Loss: 0.6616458197434744
Epoch 4/10, Average Loss: 0.6401479840278625
Epoch 5/10, Average Loss: 0.6048664748668671
Epoch 6/10, Average Loss: 0.5750219722588857
Epoch 7/10, Average Loss: 0.5199529230594635
Epoch 8/10, Average Loss: 0.3921249657869339
Epoch 9/10, Average Loss: 0.31577322135368985
Epoch 10/10, Average Loss: 0.3450445532798767
Epoch 1/10, Average Loss: 8.960730036099752
Epoch 2/10, Average Loss: 4.675712039073308
Epoch 3/10, Average Loss: 3.237902889649073
Epoch 4/10, Average Loss: 2.410461038351059
Epoch 5/10, Average Loss: 2.0012784401575723
Epoch 6/10, Average Loss: 1.2742049147685368
Epoch 7/10, Average Loss: 0.7574410090843836
Epoch 8/10,

Epoch 5/11, Average Loss: 0.6167994638284048
Epoch 6/11, Average Loss: 0.48686591784159344
Epoch 7/11, Average Loss: 0.2806057979663213
Epoch 8/11, Average Loss: 0.31497786939144135
Epoch 9/11, Average Loss: 0.47127924611171085
Epoch 10/11, Average Loss: 0.23856201767921448
Epoch 11/11, Average Loss: 0.25133806467056274
Epoch 1/11, Average Loss: 0.721810777982076
Epoch 2/11, Average Loss: 0.6663776338100433
Epoch 3/11, Average Loss: 0.6211559871832529
Epoch 4/11, Average Loss: 0.49816911419232685
Epoch 5/11, Average Loss: 0.2934209356705348
Epoch 6/11, Average Loss: 0.37179528176784515
Epoch 7/11, Average Loss: 0.2920868396759033
Epoch 8/11, Average Loss: 0.18187039718031883
Epoch 9/11, Average Loss: 0.36240923404693604
Epoch 10/11, Average Loss: 0.2394137978553772
Epoch 11/11, Average Loss: 0.16846050197879472
Epoch 1/12, Average Loss: 0.6785685221354166
Epoch 2/12, Average Loss: 0.6650881071885427
Epoch 3/12, Average Loss: 0.6591516335805258
Epoch 4/12, Average Loss: 0.63889202475547

In [25]:
print("Best Combination:", best_combination)
print("Best Accuracy:", best_accuracy)

Best Combination: {'Hidden Layer Sizes': [25, 31], 'Activation Function': <function tanh at 0x7fedbdc2fdc0>, 'Learning Rate': 0.013476052972602692, 'Batch Size': 128, 'Number of Epochs': 11}
Best Accuracy: tensor(0.9302)


In [26]:
results_df_local

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time
0,"[26, 31]",relu,0.002037,32,10,0.860465,0.123011
1,"[25, 32]",tanh,0.001000,128,11,0.813953,0.074693
2,"[25, 30]",tanh,0.010325,64,11,0.883721,0.085520
3,"[24, 31]",tanh,0.013667,128,12,0.860465,0.080261
4,"[24, 30]",tanh,0.006498,64,10,0.813953,0.075013
5,"[25, 29]",sigmoid,0.009676,128,10,0.883721,0.076302
6,"[24, 31]",relu,0.011641,64,10,0.674419,0.082256
7,"[25, 31]",sigmoid,0.017077,128,12,0.883721,0.236072
8,"[26, 31]",sigmoid,0.004359,32,10,0.860465,0.162589
9,"[26, 31]",sigmoid,0.009776,128,11,0.837209,0.173600


### Testing Local search over all datasets

In [31]:
datasets = {'wine_quality': wine_quality, 'cong_voting': cong_voting, 'bank_marketing': bank_marketing}

initial_configuration = {
    'Hidden Layer Sizes': [25, 30],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 32,
    'Number of Epochs': 10
}

param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

num_iterations = 50


all_local_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))

    best_combination, best_accuracy, results_df_local = local_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)
    
    results_df_local['dataset'] = dataset_name
    all_local_results.append(results_df_local)

Epoch 1/9, Average Loss: 3.8431643488948333
Epoch 2/9, Average Loss: 1.4305580042622572
Epoch 3/9, Average Loss: 1.3148746238164375
Epoch 4/9, Average Loss: 1.260378381591633
Epoch 5/9, Average Loss: 1.2192782255038161
Epoch 6/9, Average Loss: 1.1942444813032091
Epoch 7/9, Average Loss: 1.174361471018177
Epoch 8/9, Average Loss: 1.162521377297267
Epoch 9/9, Average Loss: 1.1497044193964063
Epoch 1/9, Average Loss: 1.3100310300756817
Epoch 2/9, Average Loss: 1.2906167324335298
Epoch 3/9, Average Loss: 1.2879267813969244
Epoch 4/9, Average Loss: 1.2874820737019639
Epoch 5/9, Average Loss: 1.2884819551479596
Epoch 6/9, Average Loss: 1.2848143950561803
Epoch 7/9, Average Loss: 1.2840711478075366
Epoch 8/9, Average Loss: 1.2931802653096205
Epoch 9/9, Average Loss: 1.2864461369309688
Epoch 1/8, Average Loss: 1.313474523509207
Epoch 2/8, Average Loss: 1.277455811851595
Epoch 3/8, Average Loss: 1.2475995377528888
Epoch 4/8, Average Loss: 1.2191014681125711
Epoch 5/8, Average Loss: 1.2068341370

Epoch 6/11, Average Loss: 1.221164917287651
Epoch 7/11, Average Loss: 1.1958898353430392
Epoch 8/11, Average Loss: 1.1870769657240323
Epoch 9/11, Average Loss: 1.1818812873465883
Epoch 10/11, Average Loss: 1.1650613363535127
Epoch 11/11, Average Loss: 1.1540190215491077
Epoch 1/10, Average Loss: 1.312894266075883
Epoch 2/10, Average Loss: 1.2884146604069904
Epoch 3/10, Average Loss: 1.2831006642499585
Epoch 4/10, Average Loss: 1.2860084208974079
Epoch 5/10, Average Loss: 1.2832164084253135
Epoch 6/10, Average Loss: 1.2813284579961577
Epoch 7/10, Average Loss: 1.280160309346907
Epoch 8/10, Average Loss: 1.2769951491268134
Epoch 9/10, Average Loss: 1.2769994413925827
Epoch 10/10, Average Loss: 1.2730077848844001
Epoch 1/10, Average Loss: 1.3129924733214584
Epoch 2/10, Average Loss: 1.2667408746444375
Epoch 3/10, Average Loss: 1.2252230845346042
Epoch 4/10, Average Loss: 1.1838363478520164
Epoch 5/10, Average Loss: 1.167256451457556
Epoch 6/10, Average Loss: 1.1799614674474563
Epoch 7/10,

Epoch 6/9, Average Loss: 1.189480486823006
Epoch 7/9, Average Loss: 1.1784316118509492
Epoch 8/9, Average Loss: 1.1698592733020432
Epoch 9/9, Average Loss: 1.160694556733582
Epoch 1/11, Average Loss: 1.3100774712357783
Epoch 2/11, Average Loss: 1.2787829849617613
Epoch 3/11, Average Loss: 1.2721528253672314
Epoch 4/11, Average Loss: 1.2638496094686122
Epoch 5/11, Average Loss: 1.2648569806221803
Epoch 6/11, Average Loss: 1.2585798750625798
Epoch 7/11, Average Loss: 1.253857074339697
Epoch 8/11, Average Loss: 1.247415201795613
Epoch 9/11, Average Loss: 1.2456097566276971
Epoch 10/11, Average Loss: 1.24555749030201
Epoch 11/11, Average Loss: 1.2244665377710495
Epoch 1/11, Average Loss: 3.3258770060685516
Epoch 2/11, Average Loss: 1.301771535098187
Epoch 3/11, Average Loss: 1.2719593903769744
Epoch 4/11, Average Loss: 1.2631182838802688
Epoch 5/11, Average Loss: 1.247205755827617
Epoch 6/11, Average Loss: 1.2427005504537945
Epoch 7/11, Average Loss: 1.2331659208777492
Epoch 8/11, Average 

Epoch 2/10, Average Loss: 17.795306662718456
Epoch 3/10, Average Loss: 19.1706379254659
Epoch 4/10, Average Loss: 8.434244950612387
Epoch 5/10, Average Loss: 3.6447204450766244
Epoch 6/10, Average Loss: 0.9648527850707372
Epoch 7/10, Average Loss: 1.5400802505513032
Epoch 8/10, Average Loss: 1.4228732076783974
Epoch 9/10, Average Loss: 0.6141952176888784
Epoch 10/10, Average Loss: 0.4859263536830743
Epoch 1/8, Average Loss: 0.6835067967573801
Epoch 2/8, Average Loss: 0.6718613107999166
Epoch 3/8, Average Loss: 0.6620963513851166
Epoch 4/8, Average Loss: 0.6535981496175131
Epoch 5/8, Average Loss: 0.6193857292334238
Epoch 6/8, Average Loss: 0.5258714854717255
Epoch 7/8, Average Loss: 0.39172091086705524
Epoch 8/8, Average Loss: 0.3297174274921417
Epoch 1/9, Average Loss: 0.6897677580515543
Epoch 2/9, Average Loss: 0.6763772467772166
Epoch 3/9, Average Loss: 0.6574471294879913
Epoch 4/9, Average Loss: 0.633711596330007
Epoch 5/9, Average Loss: 0.5925305585066477
Epoch 6/9, Average Loss: 

Epoch 3/9, Average Loss: 5.369337340195973
Epoch 4/9, Average Loss: 10.278411706288656
Epoch 5/9, Average Loss: 6.363197366396586
Epoch 6/9, Average Loss: 3.1306534508864083
Epoch 7/9, Average Loss: 2.0192471047242484
Epoch 8/9, Average Loss: 2.118045965830485
Epoch 9/9, Average Loss: 1.420587072769801
Epoch 1/9, Average Loss: 9.353139832615852
Epoch 2/9, Average Loss: 4.362604200839996
Epoch 3/9, Average Loss: 2.6036248008410134
Epoch 4/9, Average Loss: 1.4369651128848393
Epoch 5/9, Average Loss: 0.9102330654859543
Epoch 6/9, Average Loss: 0.9046085278193156
Epoch 7/9, Average Loss: 0.7607760181029638
Epoch 8/9, Average Loss: 0.5475937326749166
Epoch 9/9, Average Loss: 0.6020364711682001
Epoch 1/8, Average Loss: 0.708149919907252
Epoch 2/8, Average Loss: 0.6779329280058543
Epoch 3/8, Average Loss: 0.6647112866242727
Epoch 4/8, Average Loss: 0.6622345248858134
Epoch 5/8, Average Loss: 0.6632353961467743
Epoch 6/8, Average Loss: 0.6635582546393076
Epoch 7/8, Average Loss: 0.662759820620

Epoch 1/9, Average Loss: 0.3274146421731097
Epoch 2/9, Average Loss: 0.3215793428128784
Epoch 3/9, Average Loss: 0.3209684944919591
Epoch 4/9, Average Loss: 0.32067563668327426
Epoch 5/9, Average Loss: 0.3205187619094131
Epoch 6/9, Average Loss: 0.32041738317574114
Epoch 7/9, Average Loss: 0.32037100037469446
Epoch 8/9, Average Loss: 0.32034149388375793
Epoch 9/9, Average Loss: 0.32031971242242646
Epoch 1/9, Average Loss: 0.3265703926025664
Epoch 2/9, Average Loss: 0.32127177290690756
Epoch 3/9, Average Loss: 0.3207945376128248
Epoch 4/9, Average Loss: 0.3205624809542906
Epoch 5/9, Average Loss: 0.32027341047826324
Epoch 6/9, Average Loss: 0.3201487270880093
Epoch 7/9, Average Loss: 0.320095761330093
Epoch 8/9, Average Loss: 0.3200575681927713
Epoch 9/9, Average Loss: 0.32001838707952823
Epoch 1/10, Average Loss: 0.3269036680242969
Epoch 2/10, Average Loss: 0.32095094170529864
Epoch 3/10, Average Loss: 0.32065722877250136
Epoch 4/10, Average Loss: 0.3204688943560841
Epoch 5/10, Average

Epoch 5/10, Average Loss: 0.8055736296598006
Epoch 6/10, Average Loss: 0.7379087177144393
Epoch 7/10, Average Loss: 0.6061681968440521
Epoch 8/10, Average Loss: 0.6521289614896253
Epoch 9/10, Average Loss: 0.567654015205575
Epoch 10/10, Average Loss: 0.5035540076022337
Epoch 1/11, Average Loss: 0.32797421982276787
Epoch 2/11, Average Loss: 0.32524856855206696
Epoch 3/11, Average Loss: 0.32559417408068203
Epoch 4/11, Average Loss: 0.3272179876499384
Epoch 5/11, Average Loss: 0.32773694132716913
Epoch 6/11, Average Loss: 0.3278766795558837
Epoch 7/11, Average Loss: 0.3278544814841261
Epoch 8/11, Average Loss: 0.3280158472509639
Epoch 9/11, Average Loss: 0.327992070346926
Epoch 10/11, Average Loss: 0.32793078743717047
Epoch 11/11, Average Loss: 0.32796018195962445
Epoch 1/11, Average Loss: 0.35233256330044527
Epoch 2/11, Average Loss: 0.35095189818479483
Epoch 3/11, Average Loss: 0.3507997741398302
Epoch 4/11, Average Loss: 0.3507255930634378
Epoch 5/11, Average Loss: 0.3505769786180802
E

Epoch 5/9, Average Loss: 0.32065155564002623
Epoch 6/9, Average Loss: 0.3205857201904348
Epoch 7/9, Average Loss: 0.3205241394563786
Epoch 8/9, Average Loss: 0.32050461401638475
Epoch 9/9, Average Loss: 0.32047129008665826
Epoch 1/10, Average Loss: 14.653202865624655
Epoch 2/10, Average Loss: 2.1469431529796115
Epoch 3/10, Average Loss: 0.8291786447627901
Epoch 4/10, Average Loss: 0.5983824135957128
Epoch 5/10, Average Loss: 0.4048734764900235
Epoch 6/10, Average Loss: 0.3329298757233666
Epoch 7/10, Average Loss: 0.3089258012551706
Epoch 8/10, Average Loss: 0.30517736114561556
Epoch 9/10, Average Loss: 0.3081282249715143
Epoch 10/10, Average Loss: 0.3101339862528212
Epoch 1/9, Average Loss: 0.33266852241986006
Epoch 2/9, Average Loss: 0.3300087298872401
Epoch 3/9, Average Loss: 0.33021526618779284
Epoch 4/9, Average Loss: 0.33010619554704834
Epoch 5/9, Average Loss: 0.3303036143008656
Epoch 6/9, Average Loss: 0.3303766581835677
Epoch 7/9, Average Loss: 0.3304472791880948
Epoch 8/9, Ave

In [37]:
all_local_results_df = pd.concat(all_local_results, ignore_index=True)
all_local_results_df = pd.DataFrame(all_local_results_df)


In [38]:
all_local_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time,dataset
0,"[25, 31]",relu,0.010922,64,9,0.491503,2.248944,wine_quality
1,"[26, 32]",tanh,0.009199,128,9,0.457516,1.986409,wine_quality
2,"[25, 30]",sigmoid,0.014234,64,8,0.458824,1.450799,wine_quality
3,"[24, 32]",relu,0.015301,32,9,0.453595,1.838678,wine_quality
4,"[26, 30]",sigmoid,0.004087,128,8,0.479739,1.357335,wine_quality
...,...,...,...,...,...,...,...,...
145,"[26, 32]",sigmoid,0.006047,64,10,0.883103,9.624138,bank_marketing
146,"[25, 32]",relu,0.002728,128,10,0.883952,9.598322,bank_marketing
147,"[24, 31]",tanh,0.005917,32,11,0.892814,13.795008,bank_marketing
148,"[26, 30]",sigmoid,0.006170,64,11,0.883103,10.614253,bank_marketing


In [39]:
all_local_results_df.to_csv('local_search_results.csv', index=False)

In [40]:
top_models_rows = []

for dataset in all_local_results_df['dataset'].unique():
    top_models_rows.extend(all_local_results_df[all_local_results_df['dataset'] == dataset].nlargest(2, 'Accuracy').iterrows())

top_models_rows_data = [row[1] for row in top_models_rows]

top_models_df = pd.DataFrame(top_models_rows_data).reset_index(drop=True)

top_models_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time,dataset
0,"[25, 29]",relu,0.014104,32,11,0.503268,1.723489,wine_quality
1,"[26, 30]",relu,0.017093,128,10,0.496732,1.603845,wine_quality
2,"[26, 28]",tanh,0.003161,32,9,0.906977,0.069246,cong_voting
3,"[26, 27]",relu,0.003553,128,8,0.906977,0.052717,cong_voting
4,"[25, 31]",relu,0.005087,64,10,0.894149,9.912437,bank_marketing
5,"[25, 32]",relu,0.005360,128,10,0.893299,9.719704,bank_marketing
